In [1]:
from cleanText import text_clean, clearColumns
from inputData import textTransfer
from standardAddress import addressList, matchAddress, loadAddressList, checkValidAddressElement
from validAddress import ValidAddress

matchAddress = ValidAddress()

provinceName = "./data/provinceLib.csv"

standardAddressList = [text_clean(item) for item in addressList(provinceName)]
listAddress = loadAddressList(provinceName)

# replace the fail characteristic and the unexpected location words
def replaceCharList(text):
    replaceList = [".,", "-,", ",,","--",".-","VIET NAM","VN", "VIETNAM",",VIET NAM",",VN", ",VIETNAM",", VIET NAM",", VN", ", VIETNAM"]
    textResults = text_clean(text)
    for item in replaceList:
        textResults = textResults.replace(item," ")
    return textResults
    
def parseAddress(item):
    if (item.find(',') != -1):
        textList = item.split(",")
    elif (item.find("-") != -1):
        textList = item.split("-")
    else:
        textList = [0]
    extractList = ["0","0","0"]
    lenList = len(textList)
    if lenList < 3:
        missedItem = textTransfer(item)
        missedCharList = [",,", ', ,', '-']
        for char in missedCharList:
            missedItem = missedItem.replace(char, ",")
        missedTextList = missedItem.split(",")
        lenListTemp = len(missedTextList)
        if lenListTemp < 3:
            extractList = ["0", "0", "0"]
        else:
            extractList = [missedTextList[lenListTemp-1],
                            missedTextList[lenListTemp-2], missedTextList[lenListTemp-3]]
    else:
        extractList = [textList[lenList-1],
                        textList[lenList-2], textList[lenList-3]]

    
    extractList = checkValidAddressElement(extractList,listAddress)    
    return clearColumns(extractList[2]) + ", "+ clearColumns(extractList[1]) + ", " + clearColumns(extractList[0])
       
def AddressParsing(testAddress):
    testAddress = testAddress
    results = replaceCharList(testAddress)
    results = parseAddress(results)
    result = matchAddress.valid_address(results)
    return result

c:\Users\thanh\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
testAddress  = "đội 13, tân dân, văn giang, hưng yên"
print(AddressParsing(testAddress))

{'province': 'HUNG YEN', 'district': 'VAN GIANG', 'ward': None}
